In [2]:
import os
import io
import json
import socket
import openai
import requests
import tiktoken
from objprint import op
from chatgpter import CallChatGPT
from IPython.display import Markdown

In [3]:
with open("api_key.txt", "r") as f:
    api_key = f.read().split("\n")
    api_key = api_key[0]

# 测试API

In [ ]:
%%timeit
openai.api_key = api_key
messages = [{"role": "user", "content": "这是一次测试！"}]
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                        messages=messages,
                                        n=1,)

print(response)
print(response["usage"])
print(response["choices"][0]["message"]["content"])

In [ ]:
%%timeit
openai.api_key = api_key
messages = [{"role": "user", "content": "这是一次测试！"}]
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                        messages=messages,
                                        stream=True,)

string_io = io.StringIO()
for idx, chunk in enumerate(response):
    print(chunk)
    if chunk["choices"][0]["finish_reason"] != "stop": 
        if chunk["choices"][0]["delta"].get("content"):
            string_io.write(chunk["choices"][0]["delta"]["content"])

print(string_io.getvalue())

In [ ]:
%%timeit
messages = [{"role": "user", "content": "这是一次测试！"}] 
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})
payload = {
    "model": "gpt-3.5-turbo",
    "messages": messages,
    "temperature": 1,
    "top_p": 1,
    "n": 1,
    "stream": False,
    "presence_penalty": 0,
    "frequency_penalty": 0,
}
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
proxies = {
    "http": f"http://127.0.0.1:{7890}",
    "https": f"http://127.0.0.1:{7890}"
}
api_url = "https://api.openai.com/v1/chat/completions"
response = requests.post(api_url,
                        json=payload,
                        headers=headers,
                        timeout=200,
                        proxies=proxies,
                        stream=True,)

response = json.loads(response.text)
op(response)
op(response["usage"])
print(response["choices"][0]["message"]["content"])

In [ ]:
try:
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
except KeyError:
    tokenizer = tiktoken.get_encoding("cl100k_base")
messages = [{"role": "user", "content": "这是一次测试！"}]
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})

message = "".join([f"role: {msg['role']}, content: {msg['content']}" for msg in messages])
print(len(tokenizer.encode(message)))

In [ ]:
def is_port_in_use(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

print(is_port_in_use(7890))

# 测试模型

In [ ]:
SESSIONNUM = 1
SESSIONINDEX = 0

gpt_model = CallChatGPT(model="gpt-3.5-turbo",
                        n=SESSIONNUM)

question = "这是一次测试！"
answer_list = gpt_model(prompt=question)
display(Markdown(question))
display(Markdown(answer_list[SESSIONINDEX]))

In [ ]:
os.system(f"python -m chatgpter -ak {api_key}")